In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Transfer Learning (Image Classification Keras Model)

This notebook explains the workflow to train AI model using transfer learning with Keras.

Instructions are described in [README.md](./README.md).

## Clear cache
Clear cache including all modules, variables and history to free up RAM

In [ ]:
%reset -f
%sx sync && echo 3 | sudo tee /proc/sys/vm/drop_caches > /dev/null

## Imports

In [ ]:
import datetime
import errno
import glob
import json
import jsonschema
import os
import pathlib
import re

import numpy as np
import tensorflow as tf

%load_ext tensorboard

## Load Configurations

Load the configuration file and set the variables.

In [ ]:
def validate_symlink(path: pathlib.Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )


configuration_path = pathlib.Path("./configuration.json")
validate_symlink(configuration_path)

with open(configuration_path, "r") as f:
    app_configuration = json.load(f)

configuration_schema_path = pathlib.Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

jsonschema.validate(app_configuration, json_schema)

source_keras_model = app_configuration.get("source_keras_model", "")
if source_keras_model:
    validate_symlink(pathlib.Path(source_keras_model))

dataset_training_dir = app_configuration["dataset_training_dir"].replace(
    os.path.sep, "/"
)
validate_symlink(pathlib.Path(dataset_training_dir))

dataset_validation_dir = app_configuration["dataset_validation_dir"].replace(
    os.path.sep, "/"
)
validate_symlink(pathlib.Path(dataset_validation_dir))

evaluate_label_file = app_configuration["evaluate_label_file"].replace(os.path.sep, "/")
validate_symlink(pathlib.Path(evaluate_label_file))

batch_size = app_configuration["batch_size"]

input_tensor_size = app_configuration["input_tensor_size"]

epochs = app_configuration["epochs"]

output_dir = app_configuration["output_dir"].replace(os.path.sep, "/")
validate_symlink(pathlib.Path(output_dir))

evaluate_result_dir = app_configuration["evaluate_result_dir"].replace(os.path.sep, "/")
validate_symlink(pathlib.Path(evaluate_result_dir))

## Load Dataset

Load training/validation dataset.

In [ ]:
with open(evaluate_label_file) as f:
    labels = json.load(f)

In [ ]:
img_height = input_tensor_size
img_width = input_tensor_size

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_training_dir, image_size=(img_height, img_width), batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_validation_dir, image_size=(img_height, img_width), batch_size=batch_size
)

In [ ]:
class_names = np.array(train_ds.class_names)
num_classes = len(class_names)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)
train_ds = train_ds.map(
    lambda x, y: (normalization_layer(x), y)
)  # Where x—images, y—labels.
val_ds = val_ds.map(
    lambda x, y: (normalization_layer(x), y)
)  # Where x—images, y—labels.

In [ ]:
# If your dataset is small and you want to improve learning performance,
# please enable following lines to cache in memory and prefetch.
# AUTOTUNE = tf.data.AUTOTUNE
# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

## Transfer Learning AI Model

Load base AI Model.

In [ ]:
if not source_keras_model:
    IMG_SIZE = (input_tensor_size, input_tensor_size)
    IMG_SHAPE = IMG_SIZE + (3,)
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=IMG_SHAPE, include_top=False, weights="imagenet"
    )
else:
    if os.path.isfile(source_keras_model):
        # earlier style keras h5 file
        base_model = tf.keras.models.load_model(source_keras_model)
    else:
        # later style keras SavedModel folder
        base_model = tf.keras.models.load_model(source_keras_model)

Remove top (output) layer if needed.

In [ ]:
# If base_model includes top (output) layer, we must remove the top (output) layer. For example:
# remove_top_layer_if_needed START
# if source_keras_model:
#     top_layer_offset = -3
#     base_model = tf.keras.Model(base_model.layers[0].input,
#                                 base_model.layers[top_layer_offset].output)
# remove_top_layer_if_needed END

Create AI Model.

In [ ]:
base_model.trainable = False

inputs = tf.keras.Input(shape=(input_tensor_size, input_tensor_size, 3))

# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a number of classes
outputs = tf.keras.layers.Dense(
    num_classes,
    activation="softmax",
    kernel_regularizer=tf.keras.regularizers.l2(0.001),
)(x)

model = tf.keras.Model(inputs, outputs)

model.summary()

In [ ]:
predictions = model(image_batch)

predictions.shape

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["acc"],
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1
)  # Enable histogram computation for every epoch.

Train AI Model.

In [ ]:
history = model.fit(
    train_ds, validation_data=val_ds, epochs=epochs, callbacks=tensorboard_callback
)

Visualize training result.

By executing following cell, tensorboard starts hosting. To display the tensorboard, in VS Code PORTS tab, open the port (like 6006) in the web browser. 

In [ ]:
%tensorboard --logdir logs/fit

## Save Keras Model

In [ ]:
export_path = os.path.join(output_dir, "saved_model")

model.save(export_path)

export_path

Validate saved AI Model.

In [ ]:
# load keras model
reloaded = tf.keras.models.load_model(export_path)

In [ ]:
# validate using AI models before/after saving to file system
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

# check the result is the same (the diff is 0.0).
abs(reloaded_result_batch - result_batch).max()

In [ ]:
reloaded_predicted_id = tf.math.argmax(reloaded_result_batch, axis=-1)
reloaded_predicted_label_batch = class_names[reloaded_predicted_id]

correct_count = 0
for idx, label in enumerate(labels_batch):
    if np.int64(label) == reloaded_predicted_id[idx]:
        correct_count += 1
accuracy = correct_count / len(labels_batch)

print(accuracy)

## Evaluate Keras Model

In [ ]:
# Notes: If you want to use evaluate() method of Keras model, you can run as following:
# top_1_accuracy = reloaded.evaluate(val_ds)[1]
# print(f'\nTop1 accuracy: {top_1_accuracy}')

Enumerate images.

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text


def natural_keys(text):
    return [atoi(c) for c in re.split(r"(\d+)", text)]


files_all = sorted(
    glob.glob(f"{dataset_validation_dir}/**/*.*", recursive=True), key=natural_keys
)

folders = sorted(
    glob.glob(f"{dataset_validation_dir}/*/", recursive=True), key=natural_keys
)

# get images and ground truth for evaluation
test_images = []
ground_truth_ids = []
for folder in folders:
    files_in_folder = sorted(
        glob.glob(os.path.join(folder, "*.*"), recursive=True), key=natural_keys
    )
    for file in files_in_folder:
        label = os.path.basename(os.path.dirname(file))
        if label in labels:
            label_id = labels[label]
            filename = os.path.basename(file)
            info = dict()
            info["path"] = file
            info["imageID"] = filename
            test_images.append(info)
            ground_truth_ids.append(label_id)

Define evaluate method.

In [ ]:
def evaluate_keras_model(model, images, ground_truth):
    def load_image(image_path):
        image = tf.io.decode_jpeg(tf.io.read_file(image_path), channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        # image = tf.image.central_crop(image, central_fraction=0.875)
        image = tf.expand_dims(image, 0)
        image = tf.compat.v1.image.resize_bilinear(
            image, [input_tensor_size, input_tensor_size], align_corners=False
        )
        image = tf.squeeze(image, [0])
        return image

    image_paths = []
    for test_image in images:
        image_paths.append(test_image["path"])
    images_ds = tf.data.Dataset.from_tensor_slices(
        [str(path) for path in image_paths]
    ).map(load_image)
    labels_ds = tf.data.Dataset.from_tensor_slices(
        np.array(ground_truth).astype(np.uint32)
    )
    test_data = tf.data.Dataset.zip((images_ds, labels_ds)).shuffle(len(image_paths))

    model.trainable = False
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )
    model.summary()

    test_result = model.evaluate(test_data.batch(1))

    return test_result[1]  # Top1 accuracy

Evaluate.

In [ ]:
top_1_accuracy = evaluate_keras_model(reloaded, test_images, ground_truth_ids)
print(f"\nTop1 accuracy: {top_1_accuracy}")

Save evaluation results as **`results.json`** in **`evaluate_result_dir`**.

In [ ]:
evaluate_output_dir = pathlib.Path(evaluate_result_dir)
evaluate_output_dir.mkdir(exist_ok=True, parents=True)

with open(evaluate_output_dir / "results.json", "w") as f:
    results = dict()
    results["top_1_accuracy"] = top_1_accuracy
    json.dump(results, f, ensure_ascii=False, indent=4)

## Clear cache
Clear cache including all modules, variables and history to free up RAM

In [ ]:
%reset -f
%sx sync && echo 3 | sudo tee /proc/sys/vm/drop_caches > /dev/null